In [ ]:
from selenium import webdriver
import time

driver = webdriver.Chrome('/usr/local/bin/chromedriver')

driver.get('https://www.ancestry.com/account/signin/')
time.sleep(3)

frame = driver.find_element_by_id('signInFrame')
driver.switch_to.frame(frame)

username = driver.find_element_by_id('username')
password = driver.find_element_by_id('password')

username.send_keys("USERNAME")
password.send_keys("PASSWORD")
time.sleep(3)
driver.find_element_by_id("signInBtn").click()
time.sleep(3)

driver.get('https://www.ancestry.com/discoveryui-matches/compare/41162d3f-f9a0-4ec2-974e-c19b1ba9b248/with/cf293c4d-2665-4bd2-9a92-5450cbf30516')
time.sleep(3)

root = driver.find_element_by_xpath("//*[@class='card root gen1']")
rootHTML = root.get_attribute('innerHTML')
forename = rootHTML.split(
    '<h6 class="userCardTitle textClipped ng-star-inserted">')[1].split(
    ' <span')[0]
surname = rootHTML.split(
    f'<h6 class="userCardTitle textClipped ng-star-inserted">{forename} <span')[
    1].split('>')[1].split('</span')[0]
lifespan = rootHTML.split(
    f'<div class="userCardDates ng-star-inserted">')[
    1].split('</div>')[0]

ancestors = {}
ancestors['1'] = {}
ancestors['1']['Forename'] = forename
ancestors['1']['Surname'] = surname
ancestors['1']['Lifespan'] = lifespan
ancestors['1']['visited'] = False

root.click()
time.sleep(2)

current_key = '1'
while True:
    current_card = driver.find_elements_by_xpath(
            "//*[starts-with(@class, 'userCardTitle ng-tns-c74-0')]"
            f"[./*[contains(text(), '{forename}')]]"
            f"[./*[contains(text(), '{surname}')]]")
    if len(current_card) == 0:
        close = driver.find_element_by_xpath(
            "//*[starts-with(@class, 'closeBtn')]")
        close.click()
        root.click()
        for i in current_key[1:]:
            parent_section = driver.find_elements_by_xpath(
            "//*[starts-with(@class, 'topSpacing ng-tns-c74-0')]"
            "[./*[contains(text(), 'Parents')]]")
        
            if len(parent_section) > 0:
                parent_section = parent_section[0]
                parents = parent_section.find_elements_by_xpath(
                    ".//*[starts-with(@class, 'card ng-tns-c74-0')]")
                parents[int(i)].click()
                time.sleep(2)

    print()
    print(current_key, forename, surname)
    if ancestors[current_key]['visited'] == False:
        ancestors[current_key]['visited'] = True
        vitals = driver.find_elements_by_xpath(
            "//*[@class='con conVitals ng-tns-c74-0 ng-star-inserted']")
        if len(vitals) > 0:
            vitals = vitals[0]
            vitals = vitals.get_attribute('outerHTML').replace(
                '<div class="con conVitals ng-tns-c74-0 ng-star-inserted">'
                '<div class="ng-tns-c74-0 ng-star-inserted">', '')

            events = vitals.split('<span class="bold ng-tns-c74-0">')[1:]
            for eventHTML in events:
                event = eventHTML.split('</span>')[0]
                if event in ancestors[current_key]:
                    counter = 2
                    while True:
                        if event + f' {counter}' not in ancestors[current_key]:
                            event = event + f' {counter}'
                            break
                        else:
                            counter += 1
                ancestors[current_key][event] = {'Date': None, 'Location': None}
                if '<span class="bullet ng-tns-c74-0 ng-star-inserted"' in eventHTML: 
                    date = eventHTML.split('<br class="ng-tns-c74-0"> ')[1].split(
                        '<span class="bullet ng-tns-c74-0 ng-star-inserted"')[0]
                    location = eventHTML.split(
                        '<span class="ng-tns-c74-0 ng-star-inserted">')[1].split(
                        '</span><!----></div>')[0]
                    ancestors[current_key][event]['Date'] = date
                    ancestors[current_key][event]['Location'] = location
                else:
                    if eventHTML.startswith(
                        f'{event}</span><br class="ng-tns-c74-0">'
                        ' <!----><span class="ng-tns-c74-0 ng-star-inserted">'):
                        location = eventHTML.split(
                            '<span class="ng-tns-c74-0 ng-star-inserted">')[1].split(
                            '</span><!----></div>')[0]
                        ancestors[current_key][event]['Location'] = location
                    else:
                        date = eventHTML.split(
                            '<br class="ng-tns-c74-0"> ')[1].split(
                            '<!----><!----></div>')[0]
                        ancestors[current_key][event]['Date'] = date

        parent_section = driver.find_elements_by_xpath(
            "//*[starts-with(@class, 'topSpacing ng-tns-c74-0')]"
            "[./*[contains(text(), 'Parents')]]")
        
        if len(parent_section) > 0:
            parent_section = parent_section[0]
            parents = parent_section.find_elements_by_xpath(
                ".//*[starts-with(@class, 'card ng-tns-c74-0')]")

            counter = 0
            for parent in parents:
                parentHTML = parent.get_attribute('innerHTML')
                forename = parentHTML.split(
                    '<h6 class="userCardTitle textClipped ng-star-inserted">')[1].split(
                    ' <span')[0]
                surname = parentHTML.split(
                    f'<h6 class="userCardTitle textClipped ng-star-inserted">{forename} <span')[
                    1].split('>')[1].split('</span')[0]
                lifespan = parentHTML.split(
                    f'<div class="userCardDates ng-star-inserted">')[
                    1].split('</div>')[0]
                print('parent', counter, forename, surname)

                parent_key = current_key + str(counter)
                ancestors[parent_key] = {}
                ancestors[parent_key]['Forename'] = forename
                ancestors[parent_key]['Surname'] = surname
                ancestors[parent_key]['Lifespan'] = lifespan
                ancestors[parent_key]['visited'] = False
                counter += 1
                   
    unexplored_paths = [ancestor for ancestor in ancestors if 
                        ancestors[ancestor]['visited'] == False]
           
    if len(unexplored_paths) == 0:
        break
           
    else:
        unexplored_from_here = [path for path in unexplored_paths if 
                                path.startswith(current_key)]
    
    if len(unexplored_from_here) > 0:
        current_key = unexplored_from_here[0]
        forename = ancestors[current_key]['Forename']
        surname = ancestors[current_key]['Surname']
        lifespan = ancestors[current_key]['Lifespan']
        print('next up', forename, surname)
        
        parent_section = driver.find_elements_by_xpath(
            "//*[starts-with(@class, 'topSpacing ng-tns-c74-0')]"
            "[./*[contains(text(), 'Parents')]]")
        
        if len(parent_section) > 0:
            parent_section = parent_section[0]
            parents = parent_section.find_elements_by_xpath(
                ".//*[starts-with(@class, 'card ng-tns-c74-0')]")
        
            for parent in parents:
                parentHTML = parent.get_attribute('innerHTML')
                if (forename in parentHTML) and (
                    surname in parentHTML) and (
                    lifespan in parentHTML):
                    driver.execute_script("arguments[0].scrollIntoView();", parent)
                    parent.click()
                    time.sleep(2)
                    break
                else:
                    continue
                
    else:
        current_key = current_key[:-1]
        forename = ancestors[current_key]['Forename']
        surname = ancestors[current_key]['Surname']
        lifespan = ancestors[current_key]['Lifespan']
        print('next down', forename, surname)
        
        spouse_and_children_section = driver.find_element_by_xpath(
            "//*[starts-with(@class, 'topSpacing ng-tns-c74-0')]"
            "[./*[contains(text(), 'Spouse and Children')]]")
        spouse_and_children = spouse_and_children_section.find_elements_by_xpath(
            ".//*[starts-with(@class, 'card ng-tns-c74-0')]")
        
        for spouse_and_child in spouse_and_children:
            spouse_and_childHTML = spouse_and_child.get_attribute('innerHTML')
            if (forename in spouse_and_childHTML) and (
                surname in spouse_and_childHTML) and (
                lifespan in spouse_and_childHTML):
                driver.execute_script("arguments[0].scrollIntoView();", spouse_and_child)
                spouse_and_child.click()
                time.sleep(2)
                break
            else:
                continue